In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
pre_df = pd.read_csv('../data/cleaned/cleaned_vessel_calls.csv')

In [14]:
# preprocess dataframe - duration and month
pre_df["arrivalDate"] = pd.to_datetime(pre_df["arrivalDate"])
pre_df["sailingDate"] = pd.to_datetime(pre_df["sailingDate"])
pre_df["duration"] = pd.to_datetime(pre_df["sailingDate"]) - pd.to_datetime(pre_df["arrivalDate"])
pre_df["month"] = pre_df["arrivalDate"].dt.month

In [15]:
# preprocess dataframe - vessel type
from sklearn.preprocessing import LabelEncoder

display(pre_df["vessel.vesselType"].unique())
le_type = LabelEncoder()
pre_df["vessel.vesselType"] = le_type.fit_transform(pre_df["vessel.vesselType"])

array(['crude oil tanker', 'product tanker',
       'fully cellular containership', 'vehicle carrier',
       'Combined chemical and oil tanker', 'chemical tanker',
       'Liquefied Petroleum Gas Carrier', 'semi-sub HL vessel',
       'bulk carrier', 'general cargo with container capacity', 'tug',
       'passenger (cruise)', 'patrol ship', 'Tanker (unspecified)',
       'asphalt tanker', 'tug/supply', 'general cargo',
       'Vessel Type (unspecified)', 'Roll On Roll Off'], dtype=object)

In [16]:
# preprocess data
# sort by imo number
pre_df.sort_values(by=["IMO", "arrivalDate"], inplace=True)
pre_df['next_unlocode'] = pre_df.groupby('IMO')['place.unlocode'].shift(-1)
pre_df['is_last_imo'] = pre_df['IMO'] != pre_df['IMO'].shift(-1)
df_cleaned = pre_df[~pre_df["is_last_imo"]]

In [17]:
# encode unlocode, next unlocode
# then create X data set and Y data set
# split in test and train
# create model random forest